##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
import mediapipe as mp
import cv2
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

## Put Annotations on the frame

In [2]:
MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  return annotated_image

## Detect landmarks in vid

In [4]:
def detect_landmark_video(vid_path,out_path):
    # Use OpenCV’s VideoCapture to load the input video:
    video_capture = cv2.VideoCapture(vid_path)

    # Load the frame rate of the video using OpenCV’s CV_CAP_PROP_FPS:
    frame_rate = video_capture.get(cv2.CAP_PROP_FPS)

    # calculate the timestamp for each frame in ms:
    timestamp = 1 / frame_rate * 1000

    BaseOptions = mp.tasks.BaseOptions
    HandLandmarker = mp.tasks.vision.HandLandmarker
    HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
    VisionRunningMode = mp.tasks.vision.RunningMode

    model_path = './hand_landmarker.task'
    
    annotated_results = []
    # Create a hand landmarker instance with the video mode:
    options = HandLandmarkerOptions(
        base_options=BaseOptions(model_asset_path=model_path),
        running_mode=VisionRunningMode.VIDEO,
        min_hand_detection_confidence=0.3,
        min_hand_presence_confidence=0.4,
        min_tracking_confidence=0.4)
    with HandLandmarker.create_from_options(options) as landmarker:
        frame_timestamp = 0
        # Loop through each frame in the video using VideoCapture.read():
        while video_capture.isOpened():
            ret, frame = video_capture.read()
            if not ret:
                break
            
            # Convert the frame received from OpenCV to a MediaPipe’s Image object.
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)

            # calculate the frame timestamp in ms:
            frame_timestamp += timestamp

            # Perform hand landmarks detection on the provided single image:
            hand_landmarker_result = landmarker.detect_for_video(mp_image, int(frame_timestamp))

            annotated_image = draw_landmarks_on_image(mp_image.numpy_view(), hand_landmarker_result)
            # cv2.imshow('Frame',annotated_image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows() # It destroys the image showing the window.

            # append the annotated image to the list:
            annotated_results.append(annotated_image)

    # Release the VideoCapture object and close the OpenCV window:
    video_capture.release()

    # Save the annotated results as a video:
    height, width, _ = annotated_results[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(out_path, fourcc, frame_rate, (width, height))
    for frame in annotated_results:
        out.write(frame)

    out.release()

## 2D prediction

In [5]:
import os
import time

data_path = '/media/pruszynski/Data'
# recording_name = 'Recording_2023-05-15T154223' #Jon's no board
recording_name = 'Recording_2023-05-05T100719' #Tomo's messy situation
# recording_name = 'Recording_2024-01-31T114025' # Ali's normal recording
pred_path = './Predictions2D'

# Loop through all .avi vids in the folder
for filename in os.listdir(os.path.join(data_path, recording_name)):
    start_time = time.time()  # Record the start time of the iteration

    if filename.endswith('.avi'):
        # Construct the full path to the .avi file and output prediction:
        vid_path = os.path.join(data_path, recording_name, filename)
        out_path = os.path.join(pred_path, recording_name, filename)

        # Ensure the prediction directory exists, create it if it doesn't
        if not os.path.exists(os.path.join(pred_path,recording_name)):
            os.makedirs(os.path.join(pred_path,recording_name))
        
        # make 2D predictions:
        detect_landmark_video(vid_path,out_path)
    
    end_time = time.time()  # Record the end time of the iteration
    iteration_time = end_time - start_time  # Calculate the time taken by the iteration
    print(f"Duration: {iteration_time:.2f} seconds")


I0000 00:00:1715127415.857417  848564 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1715127415.954740  848855 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 525.105.17), renderer: NVIDIA GeForce GTX 1080 Ti/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
/home/pruszynski/Desktop/Projects/HandTracker3D/.conda/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
